In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pywt # python library for wavelet transformation

In [2]:
# An interface to calculate euclidean distance between 2 signals
def euc_dist(x,y):
    return float(np.linalg.norm(x - y))

In [3]:
import struct
filename_synthetic = 'synthetic_size50k_len256_znorm.bin'
with open(filename_synthetic, 'rb') as in_file:
    time_series_synthetic = np.array(struct.unpack('f' * 50000 * 256, in_file.read())).reshape(-1,256)

filename_seismic = 'seismic_size50k_len256_znorm.bin'
with open(filename_seismic, 'rb') as in_file:
    time_series_seismic = np.array(struct.unpack('f' * 50000 * 256, in_file.read())).reshape(-1,256)
    
filename_seismic_query = 'seismic-query_size100_len256_znorm.bin'
with open(filename_seismic_query, 'rb') as in_file:
    time_series_seismic_query = np.array(struct.unpack('f' * 100 * 256, in_file.read())).reshape(-1,256)

filename_synthetic_query = 'synthetic-query_size100_len256_znorm.bin'
with open(filename_synthetic_query, 'rb') as in_file:
    time_series_synthetic_query = np.array(struct.unpack('f' * 100 * 256, in_file.read())).reshape(-1,256)

In [4]:
def compress_to_size(x, size):
    size = int(size/4 /2)
    piece_size = int(256/size)
    com_x = np.array([], dtype = np.float32).reshape(x.shape[0], -1)
    for i in range(size):
        sum_segment = np.sum(x[:, int(piece_size*i):int(piece_size*i+piece_size-1)], axis = 1)
        abssum_segment = np.sum(np.abs(x[:, int(piece_size*i):int(piece_size*i+piece_size-1)]), axis = 1)
        com_x = np.column_stack((com_x, sum_segment))
        com_x = np.column_stack((com_x, abssum_segment))
    return com_x
def decompress1024(x, size):
    size = int(size/4 /2)
    piece_size = int(256/size)
    rec_x = np.array([], dtype = np.float32).reshape(x.shape[0], -1)
    for i in range(size):
        deviationSegment = x[:, 2*i] / piece_size
        deviationPoint = x[:, 2*i+1] / piece_size
        for j in range(int(piece_size/2)):
            m1 = deviationSegment + deviationPoint
            m2 = deviationSegment - deviationPoint
            rec_x = np.column_stack((rec_x, m1))
            rec_x = np.column_stack((rec_x, m2))
    return rec_x

In [5]:
def similarity_euclidean_compression(q, x_com, x_rec, compress_size):
    
    # Compress the query signal to a smaller time series which contains "compress_size" float numbers
    q_com = compress_to_size(q, compress_size)
    #x_com = compress_to_size(x, compress_size)
    #x_rec = decompress1024(x_com, compress_size)

    D_closest_com = 10000.0  # Initialize closest distance for compressed time series as infinitiy
    D_closest_ori = 10000.0  # Initialize closest distance for original time series as infinitiy
    index_closest = 0        # Initialize closest time series' index as 0
    pruning_count = 0        # Initialize pruning count as 0
    
    for i in range(50000): # Loop for all time_series
        
        # Calculate euclidean distance between query and compressed time series
        dist_com = euc_dist(q_com, x_com[i,]) 
        
        if D_closest_com > dist_com:
            
            # Calculate the euclidean distance between query and reconstructed time series
            dist_ori = euc_dist(q, x_rec[i,]) 
            
            if D_closest_ori > dist_ori:
                
                # Memorization
                D_closest_com = dist_com
                D_closest_ori = dist_ori
                index_closest = i
        
        # When euclidean distance between compressed time series is bigger
        else:
            # The time series is pruned, we count all pruned times
            pruning_count += 1
            
    # Return index, distance and pruning times for this similarity search
    return D_closest_ori, pruning_count

In [6]:
def similarity_euclidean_no_compression(q, time_series):

    D_closest_ori = 10000.0  # Initialize original minimum distance as infinitiy
    
    for i in range(time_series.shape[0]): # Loop for all time_series
        x = time_series[i,]
        dist_ori = euc_dist(q, x) # Calculate the euclidean distance between original time series
        
        if D_closest_ori > dist_ori:
            D_closest_ori = dist_ori
            
    # Return index, distance for this similarity search
    return D_closest_ori

In [7]:
pruning_count_128_synthetic = 0
pruning_count_64_synthetic = 0
pruning_count_32_synthetic = 0

distSimilarity_128_synthetic = []
distSimilarity_64_synthetic = []
distSimilarity_32_synthetic = []
distSimilarity_synthetic = []

x = time_series_synthetic

sum_128 = compress_to_size(x, 128)
sum_64 = compress_to_size(x, 64)
sum_32 = compress_to_size(x, 32)

rec_128 = decompress1024(sum_128, 128)
rec_64 = decompress1024(sum_64, 64)
rec_32 = decompress1024(sum_32, 32)

query_size = 100
for i in range(query_size):
    
    # Synthetic
    q = time_series_synthetic_query[i,]
    
    # 128
    dist, pruning_count = similarity_euclidean_compression(q, sum_128, rec_128, 128)
    distSimilarity_128_synthetic.append(dist)
    pruning_count_128_synthetic += pruning_count
    
    # 64
    dist, pruning_count = similarity_euclidean_compression(q, sum_64, rec_64, 64)
    distSimilarity_64_synthetic.append(dist)
    pruning_count_64_synthetic += pruning_count
    
    # 32
    dist, pruning_count = similarity_euclidean_compression(q, sum_32, rec_32, 32)
    distSimilarity_32_synthetic.append(dist)
    pruning_count_32_synthetic += pruning_count
    
    # without data summarization
    dist_no_com = similarity_euclidean_no_compression(q, time_series_synthetic)
    distSimilarity_synthetic.append(dist_no_com)

IndexError: too many indices for array

In [ ]:
pruning_count_128_seismic = 0
pruning_count_64_seismic = 0
pruning_count_32_seismic = 0

distSimilarity_128_seismic = []
distSimilarity_64_seismic = []
distSimilarity_32_seismic = []
distSimilarity_seismic = []

x = time_series_seismic

sum_128 = compress_to_size(x, 128)
sum_64 = compress_to_size(x, 64)
sum_32 = compress_to_size(x, 32)

rec_128 = decompress1024(sum_128, 128)
rec_64 = decompress1024(sum_64, 64)
rec_32 = decompress1024(sum_32, 32)

query_size = 100
for i in range(query_size):
    # Seismic
    q = time_series_seismic_query[i,]
    
    # 128
    dist, pruning_count = similarity_euclidean_compression(q, sum_128, rec_128, 128)
    distSimilarity_128_seismic.append(dist)
    pruning_count_128_seismic += pruning_count
    
    # 64
    dist, pruning_count = similarity_euclidean_compression(q, sum_64, rec_64, 64)
    distSimilarity_64_seismic.append(dist)
    pruning_count_64_seismic += pruning_count
    
    # 32
    dist, pruning_count = similarity_euclidean_compression(q, sum_32, rec_32, 32)
    distSimilarity_32_seismic.append(dist)
    pruning_count_32_seismic += pruning_count
    
    # without data summarization
    dist_no_com = similarity_euclidean_no_compression(q, time_series_seismic)
    distSimilarity_seismic.append(dist_no_com)

In [ ]:
size = 100

print("Similarity search for synthetic time series:")
print("Average pruning ratio 128/64/32: ", pruning_count_128_synthetic/size/50000, 
      pruning_count_64_synthetic/size/50000, pruning_count_32_synthetic/size/50000)

print("Similarity search for seismic time series:")
print("Average pruning ratio 128/64/32: ", pruning_count_128_seismic/size/50000, 
      pruning_count_64_seismic/size/50000, pruning_count_32_seismic/size/50000)

In [ ]:
def summarize_data(x):
    print("Average:", np.mean(x).round(4), " Mean:", np.median(x).round(4), " Max:", np.max(x).round(4), " Min:", np.min(x).round(4), "\n")

In [ ]:
print("Synthetic time series: ")
print("---------------------")
print("Similarity search error - with data summarization (128)")
summarize_data(distSimilarity_128_synthetic)
print("Similarity search error - with data summarization (64)")
summarize_data(distSimilarity_64_synthetic)
print("Similarity search error - with data summarization (32)")
summarize_data(distSimilarity_32_synthetic)
print("Similarity search error - without data summarization")
summarize_data(distSimilarity_synthetic)

In [ ]:
print("Seismic time series: ")
print("---------------------")
print("Similarity search error - with data summarization (128)")
summarize_data(distSimilarity_128_seismic)
print("Similarity search error - with data summarization (64)")
summarize_data(distSimilarity_64_seismic)
print("Similarity search error - with data summarization (32)")
summarize_data(distSimilarity_32_seismic)
print("Similarity search error - without data summarization")
summarize_data(distSimilarity_seismic)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
plt.subplot(1,2,1)
plt.boxplot([distSimilarity_128_seismic, distSimilarity_64_seismic, distSimilarity_32_seismic, distSimilarity_seismic])
plt.title('Similarity search error - 100 seismic time series')
plt.xticks([1, 2, 3, 4], ['128', '64', '32', 'no_compression'])

plt.subplot(1,2,2)
plt.boxplot([distSimilarity_128_synthetic, distSimilarity_64_synthetic, distSimilarity_32_synthetic, distSimilarity_synthetic])
plt.title('Similarity search error - 100 synthetic time series')
plt.xticks([1, 2, 3, 4], ['128', '64', '32', 'no_compression'])

plt.tight_layout()
plt.show